# Generate index weights open dedicated service - F#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index weights sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns index components and weights.

### Services used
This sample uses *gRPC requests* in order to retrieve index components and weights from the hosted service. The queried endpoint in this script are:
* *ComponentsService*: to directly retrieve index data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Indices.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run Index weights sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: CsvHelper"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Indices.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
// Import/export csv files
open CsvHelper
open System.IO
open System.Globalization
// Plot graphs
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Request creation

The following code snippet enables to select the **index** by its *Name*:

In [ ]:
// Set index name
let index = "nasdaq 100"

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
// Instantiate the indice components service
let service = new ComponentsService.ComponentsServiceClient(channel)

// Generate the indice components request
let request = new ComponentsRequest( Identifier = index )

// Process the indice components response
let reply = service.Components(request, headers)

// Display the results
display(reply.Components)

### Step 4: Retrieve index data

#### 4.1 Export index data

The following code snippet exports index data to a *csv file*:

In [ ]:
let write instruments =
    let writeRecords (csv : CsvWriter) i =
        csv.WriteRecords(i)
    let write' (writer : StreamWriter )  i =
        using (new CsvWriter(writer, CultureInfo.InvariantCulture)) (fun csv -> writeRecords csv i )
    using (new StreamWriter($"{index}.csv")) (fun writer -> write' writer instruments)


reply.Components
|> write

#### 4.2 Visualize index data

The following code snippet displays index components and weights. <br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
reply.Components
|> Seq.map (fun elt -> (elt.Identifier.Ticker, elt.Weight))
|> Chart.Pie
|> Chart.WithTitle $"{index} components and weights"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display

